In [1]:
import os

In [2]:
base_config = {
    "saudio_option": {
        "input_port": "UGREEN CM564 USB Audio  Mono:capture_MONO",
        "output_port": "USB2.0 Device Analog Stereo:playback_FL",
        "ringbuffer_size": 1280
    },
    "ofdm_option": {
        "symbol_freq": 1000,
        "symbol_bits": 1,
        "channels": [
            3,
            7
        ],
        "shift_samples": 0
    },
    "sphy_option": {
        "bin_payload_size": 40,
        "frame_gap_size": 48
    }
}

In [ ]:
import copy
import json
import itertools

config_result_mp = {}

In [18]:
freqs = [3000, 4000, 5000, 6000]
max_freq = 12000


for freq in freqs:
    channels = [i for i in range(1, 13) if i * freq <= max_freq]
    channel_tuples = itertools.combinations(channels, 2)

    for channel in channel_tuples:
        import copy
        config = copy.deepcopy(base_config)
        config["ofdm_option"]["symbol_freq"] = freq
        config["ofdm_option"]["channels"] = list(channel)
        import json
        # write to /home/ryo/code/supersonic/build/config.json
        json.dump(config, open("/home/ryo/code/supersonic/build/config.json", "w"), indent=4)

        print(f"freq: {freq}, channel: {channel}")

        os.system("cd /home/ryo/code/supersonic/build && rm input.txt")
        os.system("cd /home/ryo/code/supersonic/build && rm output.txt")
        os.system("cd /home/ryo/code/supersonic/build && ./supersonic/proj1 -t 1 > /dev/null 2> /dev/null")
        os.system("cd /home/ryo/code/supersonic/build && python ../jupyter/mdiff.py > result.txt")

        with open("/home/ryo/code/supersonic/build/result.txt", "r") as f:
            result = f.read()
            config_result_mp[(freq, channel)] = result
            print(f"freq: {freq}, channel: {channel}, result: {result}")

freq: 3000, channel: (1, 2)
freq: 3000, channel: (1, 2), result: 0.794
error

freq: 3000, channel: (1, 3)
freq: 3000, channel: (1, 3), result: 0.9

freq: 3000, channel: (1, 4)
freq: 3000, channel: (1, 4), result: 0.538
error

freq: 3000, channel: (2, 3)
freq: 3000, channel: (2, 3), result: 0.88

freq: 3000, channel: (2, 4)
freq: 3000, channel: (2, 4), result: 0.904

freq: 3000, channel: (3, 4)
freq: 3000, channel: (3, 4), result: 0.994

freq: 4000, channel: (1, 2)
freq: 4000, channel: (1, 2), result: 0.612
error

freq: 4000, channel: (1, 3)
freq: 4000, channel: (1, 3), result: 0.488
error

freq: 4000, channel: (2, 3)
freq: 4000, channel: (2, 3), result: 0.9299999999999999

freq: 5000, channel: (1, 2)
freq: 5000, channel: (1, 2), result: 0.884

freq: 6000, channel: (1, 2)
freq: 6000, channel: (1, 2), result: 0.358
error



In [19]:
def parse_output(output):
    output = output.strip()
    lines = output.split("\n")
    correct_rate = float(lines[0])
    has_error = 'error' in output
    return correct_rate, has_error

config_result_list = [(k, v) for k, v in config_result_mp.items()]

sorted_config_result_list = sorted(config_result_list, key=lambda x: parse_output(x[1])[0], reverse=True)

for k, v in sorted_config_result_list:
    correct_rate, has_error = parse_output(v)
    print(k, correct_rate, has_error)

(1000, (4, 9)) 1.0 False
(1000, (5, 7)) 1.0 False
(1000, (7, 8)) 1.0 False
(1000, (7, 9)) 1.0 False
(1000, (8, 9)) 1.0 False
(1000, (3, 7)) 0.996 False
(1000, (4, 5)) 0.996 False
(1000, (3, 5)) 0.994 False
(1000, (4, 7)) 0.994 False
(3000, (3, 4)) 0.994 False
(1000, (5, 9)) 0.99 False
(1000, (3, 4)) 0.986 False
(1000, (6, 9)) 0.986 False
(1000, (3, 9)) 0.982 False
(1000, (4, 8)) 0.97 False
(1000, (5, 6)) 0.968 False
(1000, (6, 7)) 0.964 False
(1000, (6, 8)) 0.96 False
(1000, (3, 6)) 0.946 False
(4000, (2, 3)) 0.9299999999999999 False
(1000, (4, 6)) 0.922 False
(2000, (2, 4)) 0.922 False
(1000, (5, 8)) 0.918 False
(3000, (2, 4)) 0.904 False
(3000, (1, 3)) 0.9 False
(2000, (3, 4)) 0.896 False
(5000, (1, 2)) 0.884 False
(3000, (2, 3)) 0.88 False
(2000, (2, 3)) 0.842 False
(3000, (1, 2)) 0.794 True
(1000, (3, 8)) 0.746 True
(1000, (1, 3)) 0.74 True
(1000, (1, 5)) 0.63 True
(4000, (1, 2)) 0.612 True
(1000, (1, 7)) 0.5800000000000001 True
(1000, (1, 6)) 0.542 True
(3000, (1, 4)) 0.538 True
(